# CSV Lazy Parser - NYC parking Tickets

## Goal 1
Create a Lazy iterator that parse the .csv file and store each row in a namedtuple

In [1]:
file = 'nyc_parking_tickets_extract.csv'

In [2]:
# navigate to data folder
import os

os.chdir('./Project_CSV_Lazy_Parser_data')

First, lets pars the columns header and clean them

In [3]:
with open(file) as f:
    header = next(f).strip('\n').split(',')
    header = [h.replace(' ', '_').lower() for h in header]

In [4]:
header

['summons_number',
 'plate_id',
 'registration_state',
 'plate_type',
 'issue_date',
 'violation_code',
 'vehicle_body_type',
 'vehicle_make',
 'violation_description']

Now we can create our namedTuple that we'll have the element of header as keyword

In [4]:
from collections import namedtuple
Ticket =  namedtuple('Ticket', header)

Now we can read the rest of the data

In [5]:
def read_data():
    with open(file) as f:
        next(f) # skip the header
        yield from f

In [6]:
raw_data = read_data()

In [7]:
next(raw_data)

'4006478550,VAD7274,VA,PAS,10/5/2016,5,4D,BMW,BUS LANE VIOLATION\n'

Then we are going to assign a data type to each column. To do this we have to define a specific cleaning function for each data type we expect to encounter. We also need to handle possible exception if case a value is missing or is in a different format than expected    

In [24]:
def parse_int(value, *, default=None):
    try:
        cleaned = int(value)
        return cleaned
    except ValueError:
        return default
    
def parse_str(value, *, default=None):
    try:
        cleaned = value.strip()
        if not cleaned:
            return default
        else:
            return cleaned
    except ValueError:
        return default
    
def parse_date(value, *, default=None):
    from datetime import datetime
    format_ = '%m/%d/%Y'
    try:
        return datetime.strptime(value, format_).date()
    except ValueError:
        return default    

Now we have to hard code the association between the parsing function and the columns data types. To be able to specify different defalts values we need to use a partial function (or a lambda) different for each field. (Otherwise we can't specify the default beacuse we would essentially be calling the function but withou specifying the `value` field -> i.e. not possible)

In [36]:
from functools import partial

colums_dtype_parser = (parse_int, # summon_number
                       parse_str, # plate_id
                       lambda x: parse_str(x, default=''), # registration_state
                       partial(parse_str, default=''), # plate_type
                       parse_date, # issue_date
                       parse_int, # violation_code
                       partial(parse_str, default=''), # vehicle_body_type
                       parse_str, # vehicle_make
                       partial(parse_str, default=''), # violation description
                      )

We defaulted some values to an empty string because we want to be able to differentiate what, in our arbitrary opinion, is a mandatory field. Only the fields that will default to `None` are mandatory, and therefore we will have to check later on if any value in the row is `None`, and if it is, then we will discard the row.

Now we can create the function devoted to the row parsing

In [37]:
def parse_row(row):
    row = row.strip('\n').split(',')
    row = (func(r) for func, r in zip(colums_dtype_parser, row))
    return row
    

In [38]:
raw_data = read_data()

In [39]:
next(raw_data)

'4006478550,VAD7274,VA,PAS,10/5/2016,5,4D,BMW,BUS LANE VIOLATION\n'

In [40]:
for _ in range(5):
    row = next(raw_data)
    parsed_row = parse_row(row)
    print(list(parsed_row))

[4006462396, '22834JK', 'NY', 'COM', datetime.date(2016, 9, 30), 5, 'VAN', 'CHEVR', 'BUS LANE VIOLATION']
[4007117810, '21791MG', 'NY', 'COM', datetime.date(2017, 4, 10), 5, 'VAN', 'DODGE', 'BUS LANE VIOLATION']
[4006265037, 'FZX9232', 'NY', 'PAS', datetime.date(2016, 8, 23), 5, 'SUBN', 'FORD', 'BUS LANE VIOLATION']
[4006535600, 'N203399C', 'NY', 'OMT', datetime.date(2016, 10, 19), 5, 'SUBN', 'FORD', 'BUS LANE VIOLATION']
[4007156700, '92163MG', 'NY', 'COM', datetime.date(2017, 4, 13), 5, 'VAN', 'FRUEH', 'BUS LANE VIOLATION']


To check if no Falsy data is contained in each row (and therefore decide to discard it) we can use the `all` and `any` python methods. Basically, `all` return `True` only if all the elements in a sequence evaluate to `True`, while `any`, return `True` if at least one element evaluate to `True` 

In [43]:
all([1,2,3]), all(['', 1, 2]), any(['', 0, 1]), any(['', 0, None])

(True, False, True, False)

In particular, we want to be able to see if a `None` is inside the row, therefore being able to discard it. A clever way to do it is to create an expression that evaluate True if `None` is in the sequence

In [50]:
test1 = ['', 0, 'valid'] # we have 2 Falsy element but no one is None
test2 = ['', 0, 'valid', None] # we have 3 values that evaluate to false but we want only to catch the None
check1 = all(t is not None for t in test1)
check2 = all(t is not None for t in test2)
check1, check2

(True, False)

Now we can modidy our `parse_row` function, to check for `None` entries and to return the namedTuple `Ticked` instead of a simple list

In [53]:
def parse_row(row, *, default=None):
    row = row.strip('\n').split(',')
    row = [func(r) for func, r in zip(colums_dtype_parser, row)] 
    # we want a list because we'll iterate on it at least twice and it woudn't be possible with a generator
    if all(item is not None for item in row): # if there isn't any None value
        return Ticket(*row)
    else:
        return default
    

In [58]:
raw_data = read_data()
for _ in range(3):
    row = next(raw_data)
    parsed_row = parse_row(row)
    print(parsed_row)

Ticket(summons_number=4006478550, plate_id='VAD7274', registration_state='VA', plate_type='PAS', issue_date=datetime.date(2016, 10, 5), violation_code=5, vehicle_body_type='4D', vehicle_make='BMW', violation_description='BUS LANE VIOLATION')
Ticket(summons_number=4006462396, plate_id='22834JK', registration_state='NY', plate_type='COM', issue_date=datetime.date(2016, 9, 30), violation_code=5, vehicle_body_type='VAN', vehicle_make='CHEVR', violation_description='BUS LANE VIOLATION')
Ticket(summons_number=4007117810, plate_id='21791MG', registration_state='NY', plate_type='COM', issue_date=datetime.date(2017, 4, 10), violation_code=5, vehicle_body_type='VAN', vehicle_make='DODGE', violation_description='BUS LANE VIOLATION')


At last, need a generator function that is able to parse all the data at once

In [59]:
def parse_data():
    for row in read_data(): # read_data() is a generator itself
        parsed = parse_row(row)
        if parsed: # if it not a None but a Ticket
            yield parsed
    

In [60]:
parser = parse_data()

In [62]:
for _ in range(10):
    print(next(parser))

Ticket(summons_number=4006478550, plate_id='VAD7274', registration_state='VA', plate_type='PAS', issue_date=datetime.date(2016, 10, 5), violation_code=5, vehicle_body_type='4D', vehicle_make='BMW', violation_description='BUS LANE VIOLATION')
Ticket(summons_number=4006462396, plate_id='22834JK', registration_state='NY', plate_type='COM', issue_date=datetime.date(2016, 9, 30), violation_code=5, vehicle_body_type='VAN', vehicle_make='CHEVR', violation_description='BUS LANE VIOLATION')
Ticket(summons_number=4007117810, plate_id='21791MG', registration_state='NY', plate_type='COM', issue_date=datetime.date(2017, 4, 10), violation_code=5, vehicle_body_type='VAN', vehicle_make='DODGE', violation_description='BUS LANE VIOLATION')
Ticket(summons_number=4006265037, plate_id='FZX9232', registration_state='NY', plate_type='PAS', issue_date=datetime.date(2016, 8, 23), violation_code=5, vehicle_body_type='SUBN', vehicle_make='FORD', violation_description='BUS LANE VIOLATION')
Ticket(summons_number=4

## Goal 2
Calculate the number of violation by vehicle_make

In [65]:
res = dict()
def violation_by_vehicle(row):
    vehicle = row.vehicle_make
    if vehicle in res:
        res[vehicle] += 1
    else:
        res[vehicle] = 1

In [68]:
parser = parse_data()
for row in parser:
    violation_by_vehicle(row)
    

In [73]:
sorted(res.items(), key= lambda x : x[1], reverse=True)[:5]

[('TOYOT', 114), ('HONDA', 107), ('FORD', 106), ('CHEVR', 77), ('NISSA', 70)]

Actually, there is a more clever way to achieve the dictionary population without the if-else. Inside the `collections` module there is a particural dictionary type called `defaultdict` in which we can specify a default value that will be associated to a non-exsisting key when queried (standard dict will trow an error if asked for a non-exsisting key)

In [74]:
from collections import defaultdict
res = defaultdict(int)

def violation_by_vehicle(row):
    vehicle = row.vehicle_make
    res[vehicle] += 1


In [75]:
parser = parse_data()
for row in parser:
    violation_by_vehicle(row)

sorted(res.items(), key= lambda x : x[1], reverse=True)[:5]  

[('TOYOT', 112), ('HONDA', 106), ('FORD', 104), ('CHEVR', 76), ('NISSA', 70)]